# Average models over different epochs

In [24]:
path_models = "/Users/alberic/Desktop/Pro/RouskinLab/papers/Main_Paper/DL_paper_figures/Figure5/saved_models/models_ablation"
# read all file in path
import os
import torch
models = set([f.split('_')[0] for f in os.listdir(path_models) if os.path.isfile(os.path.join(path_models, f))])
print(models)

for model in models:
    
    weights_final = torch.load(os.path.join(path_models, model + "_epoch20.pt"), map_location=torch.device('cpu'))
    weights = []
    epochs = [16,17,18,19,20]
    for epoch in epochs: weights.append(torch.load(os.path.join(path_models, model + f"_epoch{epoch}" + ".pt"), map_location=torch.device('cpu')))

    for key in weights_final.keys():
        weights_final[key] -= weights_final[key]
        for i in range(len(weights)):
            weights_final[key] += weights[i][key]
        weights_final[key] = weights_final[key]/float(len(weights))

    torch.save(weights_final, os.path.join(path_models, model + f"_epoch{epochs}_avg.pt"))


{'firm-grass-79', 'pious-dew-104', 'royal-snowball-97', 'efficient-dream-82', 'morning-sunset-80', 'logical-darkness-92', 'robust-water-107', 'desert-paper-83', 'worldly-river-105', 'glamorous-hill-96', 'laced-water-90', 'serene-flower-81'}


## Show distribution of structure prediction performance

Reuse same format as the algo_benchmark plot of Figure 1.

Add our algorithm and hopefully show less performance gap

**Assigned to**: Alberic

Use Ploty, and a white background

In [13]:
import pandas as pd
from rouskinhf import get_dataset
import torch
import numpy as np

In [14]:
def compute_f1(pred_matrix, target_matrix, threshold=0.5):
    """
    Compute the F1 score of the predictions.

    :param pred_matrix: Predicted pairing matrix probability  (L,L)
    :param target_matrix: True binary pairing matrix (L,L)
    :return: F1 score for this RNA structure
    """

    pred_matrix = (pred_matrix > threshold).float()

    sum_pair = torch.sum(pred_matrix) + torch.sum(target_matrix)

    if sum_pair == 0:
        return 1.0
    else:
        return (2 * torch.sum(pred_matrix * target_matrix) / sum_pair).item()
    
def pairList2pairMatrix(pair_list, len_seq):
    pair_list = np.array(pair_list).astype(int)
    pairing_matrix = torch.zeros((len_seq, len_seq))

    if len(pair_list) > 0:
        pairing_matrix[pair_list[:,0], pair_list[:,1]] = 1.0
        pairing_matrix[pair_list[:,1], pair_list[:,0]] = 1.0

    return pairing_matrix

In [15]:
ground_truth = pd.DataFrame()

for test_set in ["PDB", "archiveII_blast", "lncRNA", "viral_fragments"]:
    data = get_dataset(test_set, force_download=True)
    data = pd.DataFrame(data).T[['sequence', 'structure']]
    data['dataset'] = test_set

    ground_truth = pd.concat([ground_truth, data])
    del data

PDB: Downloading dataset from HuggingFace Hub...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

PDB: Download complete. File saved at data/PDB/data.json
archiveII_blast: Downloading dataset from HuggingFace Hub...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

archiveII_blast: Download complete. File saved at data/archiveII_blast/data.json
lncRNA: Downloading dataset from HuggingFace Hub...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

lncRNA: Download complete. File saved at data/lncRNA/data.json
viral_fragments: Downloading dataset from HuggingFace Hub...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

viral_fragments: Download complete. File saved at data/viral_fragments/data.json


In [65]:
data_comparison = pd.DataFrame()

path_data = 'data/results_main'

for model in os.listdir(path_data):

    prediction = pd.read_feather(os.path.join(path_data, model))

    merged = ground_truth.reset_index().rename(columns={'index': 'reference'}).merge(prediction, on=['sequence', 'reference'], suffixes=('_true', '_pred'))

    f1s = []
    for i, row in merged.iterrows():
        f1s.append(compute_f1(torch.tensor(np.stack(row['structure_pred'])), 
                            pairList2pairMatrix(row['structure_true'], len(row['sequence'])), threshold=0.5))

    model = model.split('.feather')[0].split('_')[-1]
    print(model)
    merged['f1'] = f1s
    merged['model'] = model
    print(merged.groupby('dataset')['f1'].mean())

    data_comparison = pd.concat([data_comparison, merged[['reference', 'sequence', 'model', 'dataset', 'f1', 'structure']]])

data_comparison.loc[data_comparison['dataset']=='viral_fragments', 'dataset'] = 'viral mRNA'
data_comparison.loc[data_comparison['dataset']=='lncRNA', 'dataset'] = 'long ncRNA'
data_comparison.loc[data_comparison['dataset']=='archiveII_blast', 'dataset'] = 'archiveII'

PT+FT-primiRNA
dataset
PDB                0.900898
archiveII_blast    0.666530
lncRNA             0.399686
viral_fragments    0.696693
Name: f1, dtype: float64


KeyError: "['structure'] not in index"

In [66]:
prediction

,reference,sequence,structure
0,2N7X-2D,GGUAGUUUUGGCAUGACUCUACC,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,8S95-2D,CGCCCGGAUAGCUCAGUCGGUAGAGCAGCGGCUAAAACAGCUCUGG...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,2CD1-2D,GGAAGUCCGGUCUUCGGACCGGCUUCC,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,1WTT-2D,GGACCGGAAGGUCC,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,2NCI-2D,GGACGUUAAAAGGCUUCGGCCUACGUCC,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...
779,Alphavirus_9,UUUGGUCCCAUUGCAAGAAGUGCCUAUGGAUAGGUUCGUCAUGGAC...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
780,Alphavirus_10,ACAGAAGACGCGCUCUGCUAGAUGAAACAAAGGCGUGGUUUAGAGU...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
781,Alphavirus_12,CGUAUAGUGUACGGGAACACUACCAGUUUCCUAGAUGUGUACGUGA...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
782,Alphavirus_13,UUCGCAUUCGAGCACAGCAACCCUCCAAGAGUCGACAGUUCAUGUC...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [36]:
data_comparison

,reference,sequence,model,dataset,f1
0,2N7X-2D,GGUAGUUUUGGCAUGACUCUACC,PT+FT-primiRNA,PDB,1.000000
1,8S95-2D,CGCCCGGAUAGCUCAGUCGGUAGAGCAGCGGCUAAAACAGCUCUGG...,PT+FT-primiRNA,PDB,0.458333
2,2CD1-2D,GGAAGUCCGGUCUUCGGACCGGCUUCC,PT+FT-primiRNA,PDB,1.000000
3,1WTT-2D,GGACCGGAAGGUCC,PT+FT-primiRNA,PDB,1.000000
4,2NCI-2D,GGACGUUAAAAGGCUUCGGCCUACGUCC,PT+FT-primiRNA,PDB,0.888889
...,...,...,...,...,...
779,Alphavirus_9,UUUGGUCCCAUUGCAAGAAGUGCCUAUGGAUAGGUUCGUCAUGGAC...,PT,viral mRNA,0.362832
780,Alphavirus_10,ACAGAAGACGCGCUCUGCUAGAUGAAACAAAGGCGUGGUUUAGAGU...,PT,viral mRNA,0.656934
781,Alphavirus_12,CGUAUAGUGUACGGGAACACUACCAGUUUCCUAGAUGUGUACGUGA...,PT,viral mRNA,0.418440
782,Alphavirus_13,UUCGCAUUCGAGCACAGCAACCCUCCAAGAGUCGACAGUUCAUGUC...,PT,viral mRNA,0.702222


In [19]:
result_UFold = pd.read_feather('data/results_benchmark_UFold.feather')
result_UFold.loc[result_UFold['dataset']=='viral_fragments', 'dataset'] = 'viral mRNA'
result_UFold.loc[result_UFold['dataset']=='lncRNA', 'dataset'] = 'long ncRNA'

In [42]:
result_algos = pd.read_feather('data/results_benchmark_algos.feather')
result_algos.loc[result_algos['dataset']=='viral_fragments', 'dataset'] = 'viral mRNA'
result_algos.loc[result_algos['dataset']=='lncRNA', 'dataset'] = 'long ncRNA'
result_algos.loc[result_algos['dataset']=='archiveII_blast', 'dataset'] = 'archiveII'

In [43]:
# Creat a box plot with plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

colors = px.colors.qualitative.Pastel
    
# UFold results
for i, model in enumerate(['UFold']):
    results_dataset = result_algos[result_algos['model']==model]
    fig.add_trace(go.Violin(x=results_dataset['dataset'], y=results_dataset['F1'], 
                            name=model, marker_color=colors[i+5], 
                            meanline_visible=True, points=False))

for i, model in enumerate(['PT', 'PT+FT']):
    results_dataset = data_comparison[data_comparison['model']==model]
    fig.add_trace(go.Violin(x=results_dataset['dataset'], y=results_dataset['f1'], 
                            name=model, marker_color=colors[i], 
                            meanline_visible=True, points=False))
    
fig.update_layout(
                # title='F1 score on test sets for pretraining vs pretraining + fine-tuning models', 
                yaxis_title='F1 score', xaxis_title='Test set',
                  violinmode='group', yaxis_range=[0, 1],
                  width=1000, height=400,
                  template='plotly_white', font_size=20, font_color='black', font_family='helvetica light')
# fig.update_xaxes(categoryorder='array', categoryarray= ['RNAstructure', 'EternaFold', 'MXFold2', 'UFold'])
fig.show()

In [45]:
# save pdf
fig.write_image("images/a_structure_performance.pdf")
fig.write_image("images/a_structure_performance.eps")

## Scatter plot -> not gonna show that

In [1]:
from plotly.subplots import make_subplots

compare_f1 = data_comparison[data_comparison['model']=='PT'].merge(data_comparison[data_comparison['model']=='PT+FT'], on=['reference', 'sequence', 'dataset'], suffixes=('_PT', '_PT+FT'))[['dataset', 'f1_PT', 'f1_PT+FT']]
compare_f1['isBetter'] = compare_f1['f1_PT+FT'] >= compare_f1['f1_PT']

fig = make_subplots(rows=1, cols=3, shared_xaxes=True, shared_yaxes=True, 
                    subplot_titles=[ f'{dataset} ({100*compare_f1[compare_f1["dataset"]==dataset]["isBetter"].sum()/sum(compare_f1["dataset"]==dataset):.1f}% ≥ y=x)' for dataset in compare_f1['dataset'].unique()],
                    x_title='F1 score for PT', y_title='F1 score for PT+FT')

colors = px.colors.qualitative.D3
for i, dataset in enumerate(compare_f1['dataset'].unique()):

    data = data_comparison[data_comparison['dataset']==dataset]

    fig.add_trace(go.Scatter(x=data[data['model']=='PT']['f1'],
                        y=data[data['model']=='PT+FT']['f1'],
                        mode='markers', 
                        marker_color=colors[0], opacity=0.8, showlegend=False,
                        name=dataset), row=1, col=i+1)
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line_color='black', line_dash='dash', showlegend=False), row=1, col=i+1)


fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
  )

fig.update_layout(width=1200, height=450, template='plotly_white', font_size=15, font_color='black',
                  title='F1 score comparison per sequence for pretraining vs pretraining + fine-tuning models')
fig.show()

NameError: name 'data_comparison' is not defined